In [1]:
import redis
r=redis.StrictRedis(host='localhost',port=6379,db=0)
print('connected')


connected


In [3]:
llm_functions=[
    {
        "event_name":"quiz_generator_history",
        "latency":2,
        "token_cost":200,
        
    },
    {
        "event_name":"quiz_generator_math",
        "latency":3,
        "token_cost":1000,
        
    },
    {
        "event_name":"lesson_summary",
        "latency":5,
        "token_cost":500,
        
    }
    
    
]

api_keys=[
    
    {
        "deployment":"production_1",
        "rate_limit":4000,
        
    },
    {
        "deployment":"production_2",
        "rate_limit":8000,
        
    },
    
]
api_keys_list=[i["deployment"]for i in api_keys]
for i in api_keys:
    r.hset(i['deployment'],key=i["deployment"],value=i["rate_limit"],mapping=i)
import uuid
def generate_request_id(event_name):
    return f"{event_name}:{uuid.uuid4()}"

async def test_with_sleep(f):
    info = r.info()
    memory_usage = info['used_memory']
    print(f'Memory usage: {memory_usage} bytes')
    pass_count=0
    test_count = 10000
    sleep_total = 0
    tasks = []
    for i in range(test_count):
        event = llm_functions[random.randint(0,2)]
        tasks.append(f(i, event))
        # random_sleep_i = (random.randint(0,5))/10
        # sleep_total += random_sleep_i
        # await asyncio.sleep(random_sleep_i)
    results = await asyncio.gather(*tasks)
    sleep_total = sum(result[1] for result in results)
    print(f"task prepared, sleep {sleep_total}")
    pass_count=sum(1 for result in results if result[0])
    print(f"pass_count:{pass_count}")
    info = r.info()
    memory_usage = info['used_memory']
    print(f'Memory usage: {memory_usage} bytes')

In [3]:
import time
import random

def add_request(event_name, token_cost,latency=60):
    current_time = time.time()
    request_id = generate_request_id(event_name)
    
    r.zadd(event_name, {request_id: current_time})
    
    r.set(f"request_details:{request_id}", value=token_cost,ex=latency)
    
    return request_id, current_time

def is_request_allowed(event_name, token_limit=3000):
    min_timestamp = time.time() - 60  # 過去 60 秒
    
    r.zremrangebyscore(event_name, '-inf', min_timestamp)
    # print("??")
    
    request_ids = r.zrangebyscore(event_name, min_timestamp, 'inf')
    
    total_tokens = 0
    for req_id in request_ids:
        
        token_cost = r.get(f"request_details:{req_id.decode('utf-8')}")
        if token_cost is None:
            continue
        total_tokens += int(token_cost)
    
    return total_tokens <= token_limit



def testing():
    pass_count=0
    test_count = 10000
    for i in range(test_count):
        event = llm_functions[random.randint(0,2)]
        event_name=event['event_name']+str(i)
        token_cost=event['token_cost']
        latency=event['latency']
        
        for j in api_keys_list:
            # print(j)
            event_name=j+"_event"
            
            token_limit=int(r.hget(j,key=j))
            # print(j,token_limit)
            
            if is_request_allowed(event_name, token_limit):
                add_request(event_name, token_cost,latency)
                pass_count+=1
                print(i,"OK",j)
                # print(f"Request {request_id} added at {timestamp}, allowed.")
                break
            # else:
                # print(i,"error",j)
                # print("Request not allowed due to rate limit.")
                # error_count+=1
            # print( i , "OK")
            time.sleep((random.randint(0,5))/10)
    print(f"pass_count:{pass_count}")
    
testing()

0 OK production_1
1 OK production_1
2 OK production_1
3 OK production_1
4 OK production_1
5 OK production_1
6 OK production_1
7 OK production_1
8 OK production_1
9 OK production_2
10 OK production_2
11 OK production_2
12 OK production_2
13 OK production_2
14 OK production_2
15 OK production_2
16 OK production_2
17 OK production_2
18 OK production_1
19 OK production_2
20 OK production_2
21 OK production_2
22 OK production_1
23 OK production_1
24 OK production_1
25 OK production_1
26 OK production_2
27 OK production_2
28 OK production_2
29 OK production_2
30 OK production_2
31 OK production_1
32 OK production_1
33 OK production_1
34 OK production_2
35 OK production_2
36 OK production_2
37 OK production_2
38 OK production_1
39 OK production_1
40 OK production_1
41 OK production_1
42 OK production_2
43 OK production_2
44 OK production_2
45 OK production_2
46 OK production_2
47 OK production_2
48 OK production_2
49 OK production_2
50 OK production_1
51 OK production_2
52 OK production_1
53 

In [28]:
# v2, sliding window
import time
import random
window_s = 60

def add_request1(event, token_cost,latency=60):
    current_time = time.time()
    request_id = generate_request_id(event)
    
    r.zadd(event, {request_id: current_time})
    
    r.set(f"request_details:{request_id}", value=token_cost,ex=latency+window_s)
    
    return request_id, current_time

def is_request_allowed1(event, event_cost, token_limit=3000):
    min_timestamp = time.time() - 60  # 過去 60 秒
    
    r.zremrangebyscore(event, '-inf', min_timestamp)
    # print("??")
    
    request_ids = r.zrangebyscore(event, min_timestamp, 'inf')
    
    total_tokens = 0
    for req_id in request_ids:
        
        token_cost = r.get(f"request_details:{req_id.decode('utf-8')}")
        if token_cost is None:
            continue
        total_tokens += int(token_cost)
    total_tokens += event_cost
    
    return total_tokens <= token_limit



def testing1():
    pass_count=0
    test_count = 10000
    sleep_time = 0
    for i in range(test_count):
        event = llm_functions[random.randint(0,2)]
        # event_name=event['event_name']+str(i)
        token_cost=event['token_cost']
        latency=event['latency']
        
        for j in api_keys_list:
            # print(j)
            event_name=j+"_event"
            
            token_limit=int(r.hget(j,key=j))
            # print(j,token_cost, token_limit)
            
            if is_request_allowed1(event_name, token_cost, token_limit=token_limit):
                add_request1(event_name, token_cost,latency)
                pass_count+=1
                print(i,"OK",j)
                # print(f"Request {request_id} added at {timestamp}, allowed.")
                break
            # else:
                # print(i,"error",j)
                # print("Request not allowed due to rate limit.")
                # error_count+=1
            # print( i , "OK")
            time.sleep((random.randint(0,5))/10)
    print(f"pass_count:{pass_count}")
    
testing1()

0 OK production_1
1 OK production_1
2 OK production_1
3 OK production_1
4 OK production_1
5 OK production_1
6 OK production_1
7 OK production_1
8 OK production_2
9 OK production_2
10 OK production_2
11 OK production_2
12 OK production_2
13 OK production_2
14 OK production_2
15 OK production_2
16 OK production_2
17 OK production_2
18 OK production_2
19 OK production_2
20 OK production_2
22 OK production_2
23 OK production_2
133 OK production_1
134 OK production_1
137 OK production_1
139 OK production_1
141 OK production_1
142 OK production_1
144 OK production_2
145 OK production_2
146 OK production_1
147 OK production_1
148 OK production_1
149 OK production_1
150 OK production_2
151 OK production_1
152 OK production_2
153 OK production_2
154 OK production_2
155 OK production_2
156 OK production_2
157 OK production_2
158 OK production_2
159 OK production_2
160 OK production_2
161 OK production_2
162 OK production_2
164 OK production_2
166 OK production_2
173 OK production_2
175 OK produc

In [4]:
# token budget
import random
import asyncio
import time
from algorithm.LuaTokenBudget import TokenBudget, close_connection
from models.Models import Param, Budget
budget_models = [Budget(name=k["deployment"], limit=k["rate_limit"], interval_ms=60000) for k in api_keys]
budgets = [TokenBudget(m) for m in budget_models]

async def is_request_allowed2(req_id, llm_f):
    param = Param(req_id=req_id, amount=llm_f["token_cost"], latency=llm_f["latency"])
    total_sleep = 0
    for b in budgets:
        if b.check_and_eat(param):
            print(req_id,"OK",b.name)
            await asyncio.sleep(param.latency)  # lantency
            b.check_and_release(param.amount)
            return True, 0
        # else:
        #     print(req_id,"Failed",b.name)
        random_sleep_i = (random.randint(0,5))/10
        total_sleep += random_sleep_i
        time.sleep(random_sleep_i)
    return False, total_sleep
    
await test_with_sleep(is_request_allowed2)

Redis connection successful.
Memory usage: 1653856 bytes
0 OK production_1
1 OK production_1
2 OK production_1
3 OK production_1
4 OK production_1
5 OK production_1
6 OK production_2
7 OK production_2
8 OK production_1
9 OK production_2
10 OK production_1
11 OK production_1
12 OK production_2
13 OK production_2
14 OK production_2
15 OK production_2
16 OK production_2
17 OK production_2
18 OK production_2
19 OK production_2
20 OK production_2
21 OK production_2
23 OK production_1
24 OK production_1
25 OK production_1
26 OK production_1
27 OK production_1
28 OK production_2
31 OK production_2
37 OK production_2
38 OK production_2
39 OK production_2
40 OK production_2
41 OK production_2
42 OK production_2
43 OK production_2
44 OK production_2
45 OK production_2
46 OK production_2
47 OK production_2
48 OK production_2
49 OK production_2
50 OK production_2
51 OK production_2
52 OK production_2
62 OK production_2
63 OK production_2
64 OK production_2
65 OK production_2
66 OK production_2
67 

In [3]:
# fix window
import time
import random
import asyncio
from models.Models import Param, Budget
from algorithm.FixedWindow import FixedWindowCounter
budget_models = [Budget(name=k["deployment"], limit=k["rate_limit"], interval_ms=60000) for k in api_keys]
counters = [FixedWindowCounter(m) for m in budget_models]

async def is_request_allowed_fixed(req_id, llm_f):
    param = Param(req_id=req_id, amount=llm_f["token_cost"], latency=llm_f["latency"])
    total_sleep = 0
    for c in counters:
        if c.check_and_eat(param):
            print(req_id,"OK",c.name)
            await asyncio.sleep(param.latency)  # lantency
            return True, 0
        # else:
        #     print(req_id,"Failed",b.name)
        random_sleep_i = (random.randint(0,5))/10
        time.sleep(random_sleep_i)
        total_sleep += random_sleep_i
    return False, total_sleep


async def testing3(f):
    info = r.info()
    memory_usage = info['used_memory']
    print(f'Memory usage: {memory_usage} bytes')
    pass_count=0
    test_count = 10000
    sleep_total = 0
    tasks = []
    for i in range(test_count):
        event = llm_functions[random.randint(0,2)]
        tasks.append(f(i, event))
        # random_sleep_i = (random.randint(0,5))/10
        # sleep_total += random_sleep_i
        # await asyncio.sleep(random_sleep_i)
    results = await asyncio.gather(*tasks)
    sleep_total = sum(result[1] for result in results)
    print(f"task prepared, sleep {sleep_total}")
    pass_count=sum(1 for result in results if result[0])
    print(f"pass_count:{pass_count}")
    info = r.info()
    memory_usage = info['used_memory']
    print(f'Memory usage: {memory_usage} bytes')
    
await testing3(is_request_allowed_fixed)

0 OK production_1
1 OK production_1
2 OK production_1
3 OK production_1
4 OK production_1
5 OK production_2
6 OK production_2
7 OK production_2
8 OK production_2
9 OK production_2
10 OK production_2
11 OK production_2
12 OK production_2
13 OK production_2
14 OK production_2
15 OK production_2
16 OK production_2
44 OK production_1
45 OK production_1
46 OK production_1
47 OK production_1
48 OK production_1
49 OK production_1
50 OK production_1
51 OK production_1
52 OK production_1
53 OK production_1
54 OK production_2
55 OK production_2
56 OK production_2
57 OK production_2
58 OK production_2
59 OK production_2
60 OK production_2
61 OK production_2
62 OK production_2
63 OK production_2
64 OK production_2
65 OK production_2
66 OK production_2
67 OK production_2
68 OK production_2
69 OK production_2
70 OK production_2
71 OK production_2
184 OK production_1
185 OK production_1
186 OK production_1
187 OK production_1
188 OK production_1
189 OK production_1
190 OK production_1
191 OK producti